In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# BASIC SETUP

In [ ]:
# BASIC SETUP
! [ ! -z "$COLAB_GPU" ] && pip install torch skorch && pip install neptune-client

!cp "drive/My Drive/dl_project_data/repo/data_loading.py" .
!mkdir ./helper_scripts/
!cp "drive/My Drive/dl_project_data/repo/helper_scripts/visual_helpers.py" ./helper_scripts
!cp "drive/My Drive/dl_project_data/repo/architecture.py" .
!cp "drive/My Drive/dl_project_data/repo/model_training.py" .
!mkdir ./train/
# Creates RAM-Disk for potential speed-up
!sudo mount -t tmpfs -o size=7g tmpfs train
!for i in 0; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;
#!for i in 0 1 2 3 4 5 6 7 8 9 10 11; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;

     |████████████████████████████████| 122kB 2.6MB/s 
     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 829kB 8.2MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 204kB 12.8MB/s 
     |████████████████████████████████| 163kB 13.4MB/s 
     |████████████████████████████████| 133kB 14.0MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 143kB 14.1MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150019 sha256=91d888468a459285ad323001835c5c2238987fe936de1dc74a7e7bfe8683b175
  Stored in directory: /root/.cache/pip/wheels/84/5c/c1/a81e80761b94b4467fd3fda1fd3109463702f6247fc422eb33
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=ea475764d989f53913858dd6503fbd8f7ed3a109a499bbd7ca27d84fe4fa613f
  Stored in direc

# IMPORTS
**You should not have to change anything here.**

In [ ]:
# IMPORTS
from collections import OrderedDict

import torch
from torch import nn
from torchvision import models, transforms

import skorch.callbacks as scb
from skorch import NeuralNetBinaryClassifier

import model_training as md
import architecture as arch
from data_loading import ToTensor, Normalize, RandomRotation, RandomHorizontalFlip

# Skorch uses some depricated scikit code - works just fine, so surpress warnings for clean output
import warnings
warnings.filterwarnings('ignore')

# CLASSIFIER PARAMETRIZATION

Here you can parametrize your model and set loss, optimizer, learning rate, etc. 

For further information on what can be set and how, please refer to the [skorch documentation](https://skorch.readthedocs.io/en/stable/classifier.html#skorch.classifier.NeuralNetClassifier).

In [ ]:
# CLASSIFIER PARAMETRIZATION
classifier = NeuralNetBinaryClassifier(
    arch.ResNet18,
    optimizer = torch.optim.Adam, 
    max_epochs = 2,
    lr = 0.01,
    batch_size = 128,
    iterator_train__shuffle = True, # Shuffle training data on each epoch
    train_split = None,
    callbacks = [scb.LRScheduler(policy = 'StepLR', gamma = 0.4, step_size = 2.0)], 
    device ='cuda')

# CLASSIFIER TRAINING

After you have added the shared folder with the data to your drive as a shortcut, you should not have to change anything here. At least for now.

**IF YOU WANT TO TRAIN WITH THE FULL DATASET, JUST REMOVE** *_small* **FROM THE CSV FILE.** 

In [ ]:
# CLASSIFIER TRAINING
md.train_model(classifier, 
            train_labels = "drive/My Drive/dl_project_data/train/train_split_small.csv", 
            test_labels = "drive/My Drive/dl_project_data/train/test_split_small.csv", 
            file_dir = "train", 
            train_transform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'), # 96 + 2*64 = 224
                                  transforms.RandomHorizontalFlip(),  # TODO: model expects normalized channel values (substract means)
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20),
                                  transforms.ToTensor()]),
            test_transform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'), # 96 + 2*64 = 224
                                  transforms.ToTensor()]),
            in_memory = False,
            output_path = ".",
            #output_path = "drive/My Drive/dl_project_data/output",
            logger = {
                "api_token": "",
                "project_qualified_name": "elangenhan/hcd-experiments",
                "experiment_name": "ResNet18"
            }
)

https://ui.neptune.ai/elangenhan/hcd-experiments/e/HCDEX-58
Starting Training for <class 'architecture.ResNet18'> 
          Model-Params:
              Criterion:     <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>
              Optimizer:     <class 'torch.optim.adam.Adam'>
              Learning Rate: 0.01
              Epochs:        2
              Batch size:    128
              


  epoch    test_acc    test_f1    test_precision    test_recall    test_roc_auc    train_acc    train_f1    train_loss    train_precision    train_recall    train_roc_auc       dur
-------  ----------  ---------  ----------------  -------------  --------------  -----------  ----------  ------------  -----------------  --------------  ---------------  --------
      1      0.7419     0.7415            0.6282         0.9046          0.7670       0.7814      0.7146        0.5049             0.7481          0.6840           0.8407  129.5254


      2      0.7957     0.6948            0.8934         0.5685          0.7607       0.8215      0.7652        0.4078             0.8080          0.7266           0.8902  130.6531


Saving model...
Saving completed...
